# Modelling a non-stationary poisson process

>A non-stationary Poisson process (NSPP) is an arrival process with an arrival rate that varies by time.

One of the limitations of queuing theory is the difficulty of modelling time-dependent arrivals.  Computer 
simulation offer a number of ways of modelling non-stationary arrivals.  

In this lab you will learn:
    
* How to implement the thinning algorithm to model a non-stationary poisson process (NSPP)

> **Special thanks** to two 2020/21 students Tamir and Simon who spotted bugs in the original code!

---

# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import simpy

# please use simpy version 4
simpy.__version__

----
## An example NSPP

The table below breaks an arrival process down into 60 minutes intervals.


| t(min) | Mean time between arrivals (min) | Arrival Rate $\lambda(t)$ (arrivals/min) |
|:------:|:--------------------------------:|:--------------------------------------:|
|    0   |                15                |                  1/15                  |
|   60   |                12                |                  1/12                  |
|   120  |                 7                |                   1/7                  |
|   180  |                 5                |                   1/5                  |
|   240  |                 8                |                   1/8                  |
|   300  |                10                |                  1/10                  |
|   360  |                15                |                  1/15                  |
|   420  |                20                |                  1/20                  |
|   480  |                20                |                  1/20                  |

> **Interpretation**: In the table above the fastest arrival rate is 1/5 customers per minute or 5 minutes between customer arrivals.

## Thinning

Thinning is a acceptance-rejection sampling method and is used to generate inter-arrival times from a NSPP.  

> A NSPP has arrival rate $\lambda(t)$ where $0 \leq t \leq T$

**The thinning algorithm**

A NSPP has arrival rate $\lambda(t)$ where $0 \leq t \leq T$

Here $i$ is the arrival number and $\mathcal{T_i}$ is its arrival time.

1. Let $\lambda^* = \max_{0 \leq t \leq T}\lambda(t)$ be the maximum of the arrival rate function and set $t = 0$ and $i=1$

2. Generate $e$ from the exponential distribution with rate $\lambda^*$ and let $t = t + e$ (this is the time of the next entity will arrive)

3. Generate $u$ from the $U(0,1)$ distribution.  If $u \leq \dfrac{\lambda(t)}{\lambda^*}$ then $\mathcal{T_i} =t$ and $i = i + 1$

4. Go to Step 2.

# Exercise 1: simulation **without thinning**

**Task:**
* Build a simple `simpy` model that simulates time-dependent arrivals
* For this exercise please **IGNORE** the need for a thinning process.

**Optional task:**
* It is useful to set the sampling of arrivals using a random seed.  This will allow you to compare the number of arrivals before and after adding thinning.  **Remember that an issue with DES without thinning occurs when moving from a period $t$ with a low arrival rate to $t+1$ that has a high one.**

**Hints:**
* Build your model up gradually. 
* Start by building a model that simulates exponential arrivals using a single mean inter-arrival time then add in logic to change which mean you use depending on the simulation time.
* The logic to decide the time period is equivalent to asking yourself "given `env.now()` and that arrival rates are split into 60 minute chunks which row of my dataframe should I select".
* To simplify the task you set the run length of the simulation to no more than 540 minutes.  For an extra challenge think about how you would run the model for longer than 480 minutes and loop back round to the first period (the code to do this is surprising simple).

The data are stored in a file `data/nspp_example1.csv`. 

In [ ]:
# your code here ...

In [ ]:
# example answer

# read in the data and calculate lambda
arrivals = pd.read_csv('data/nspp_example1.csv')
arrivals['arrival_rate'] = 1 / arrivals['mean_iat']
arrivals

In [ ]:
# This example answer shows how to build up your functionality gradually.

# iteration 1: ignoring time dependence.  
# I just use the first mean in the dataframe and build up the basic structure of 
# the code. A few lines of code and have an incorrect(!) but running model.

def arrivals_generator(env, means, random_seed=None):
    rng = np.random.default_rng(random_seed)
    
    for n in itertools.count():
        interarrival_time = rng.exponential(means['mean_iat'].iloc[0])
        yield env.timeout(interarrival_time)
        print(f'arrival {n} at {env.now}')


In [ ]:
RUN_LENGTH = 540
env = simpy.Environment()
env.process(arrivals_generator(env, arrivals, random_seed=42))
env.run(RUN_LENGTH)

In [ ]:
# iteration 2.  I've now added an index t used to select the correct mean IAT.

def arrivals_generator(env, means, random_seed=None):
    rng = np.random.default_rng(random_seed)
    
    for n in itertools.count():
        
        # this give us the index of dataframe to use
        # I've used mod 9 so that run_lengh can be > 540
        t = int(env.now // 60) % 9
        interarrival_time = rng.exponential(means['mean_iat'].iloc[t])
        yield env.timeout(interarrival_time)
        print(f'arrival {n} at {env.now}')

In [ ]:
RUN_LENGTH = 540
env = simpy.Environment()
env.process(arrivals_generator(env, arrivals, random_seed=42))
env.run(RUN_LENGTH)

## Exercise 2: Thinning the arrivals

**Task:**
* Update your exercise 1 code to include an implementation of thinning
* What do you notice about the total number of arrivals compared to the previous example? Why has the changed occurred?
   * If you are not controlling your sampling with random seeds you will need to run each implementation a few times.

**Hints:**
* You will need a second distribution - Uniform(0, 1) to do the thinning.  If you are controlling random sampling through seeds that means you will need a second seed.


In [ ]:
# your code here ...

In [ ]:
# Example answer ...

# I've added an extra bit of code here to report the number of rejections for each arrival
# You wouldn't include that in production code

def arrivals_generator_with_thinning(env, means, audit=None, seed1=None, 
                                     seed2=None):
    
    arr_rng = np.random.default_rng(seed1)
    thinning_rng = np.random.default_rng(seed2)
    
    # maximum arrival rate (smallest time between arrivals)
    lambda_max = means['arrival_rate'].max()
    
    for n in itertools.count():

        # this give us the index of dataframe to use
        t = int(env.now // 60) % 9
        lambda_t = means['arrival_rate'].iloc[t]
        
        # set to a large number so that at least 1 sample taken!
        u = np.Inf
        rejects = -1
        
        interarrival_time = 0.0
        
        # reject samples if u >= lambda_t / lambda_max
        while u >= (lambda_t / lambda_max):
            rejects += 1
            interarrival_time += arr_rng.exponential(1/lambda_max)
            u = thinning_rng.uniform(0, 1)
        
        yield env.timeout(interarrival_time)
        
        # if audit included then record arrival numbers
        if audit != None: 
            audit[-1] += 1
        else:
            print(f'arrival {n} at {env.now:.2f}. Rejected samples = {rejects}')

In [ ]:
RUN_LENGTH = 540

env = simpy.Environment()
env.process(arrivals_generator_with_thinning(env, arrivals, 
                                             seed1=42, seed2=101))
env.run(RUN_LENGTH)

# Optional extra: Validate the total number of arrivals in 540 minutes.

Here we will repeat the same 10,000 times and then explore the distribution of the number of arrivals.  If all has gone to plan this should be a Poisson distribution with mean ~53.

In [ ]:
RUN_LENGTH = 540
REPLICATIONS = 10_000
audit = []

rng = np.random.default_rng(42)

for i in range(REPLICATIONS):
    # set up audit for replication.
    audit.append(0)
    env = simpy.Environment()
    # don't set the random number generator: we want different results each time
    env.process(arrivals_generator_with_thinning(env, arrivals, audit))
    env.run(RUN_LENGTH)

In [ ]:
# distribution
plt.hist(audit);

In [ ]:
# mean
np.array(audit).mean().round(2)

In [ ]:
# expected arrivals from data.
round(sum(arrivals['arrival_rate'] * 60), 2)